In [1]:
#讀檔後斷詞，並統計tf及df
#Big Data & Business Analytics, National Taiwan University

In [2]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

In [3]:
# impoort 斷詞（要放 topwords_zh.txt 進資料夾）
with open('stopwords_zh.txt', 'r') as file:
    stopwords = file.read().splitlines() 
file.close()

In [4]:
# import stock excel
# 只留台積電的日期跟收盤價
# 按照日期由最早往最晚排
df_stock = pd.DataFrame()
for i in range(2021, 2024):
    x = pd.read_excel("../../bda2023_mid_dataset/stock_data_2019-2023.xlsx", sheet_name=f'上市{i}', usecols=['證券代碼', '年月日', '收盤價(元)'])
    x = x[x['證券代碼'] == '2330 台積電']
    x = x.loc[::-1]
    df_stock = pd.concat([df_stock, x])

In [5]:
df_stock = df_stock.reset_index(drop = True)
df_stock

,證券代碼,年月日,收盤價(元)
0,2330 台積電,2021/01/04,526.7744
1,2330 台積電,2021/01/05,532.6711
2,2330 台積電,2021/01/06,539.5506
3,2330 台積電,2021/01/07,555.2752
4,2330 台積電,2021/01/08,570.0170
...,...,...,...
534,2330 台積電,2023/03/20,512.0000
535,2330 台積電,2023/03/21,517.0000
536,2330 台積電,2023/03/22,533.0000
537,2330 台積電,2023/03/23,538.0000


In [6]:
# Requirement 1 的第二點

day_n = 3 # 以 day_n 天後來看是漲還是跌
sigma = 0.01 # 以 sigma 決定漲（跌）幅超過幾 % 是漲（跌）
label = []
for i in range(len(df_stock) - day_n):
    rate = (df_stock['收盤價(元)'][i + day_n] - df_stock['收盤價(元)'][i]) / df_stock['收盤價(元)'][i]
    if rate > sigma:
       label.append('漲')
    elif rate < -sigma:
        label.append('跌')
    else:
        label.append('持平')
for i in range(day_n):
    label.append(0)
# label.extend([0, 0, 0])

df_stock['label'] = label
df_stock.tail(10)

,證券代碼,年月日,收盤價(元),label
529,2330 台積電,2023/03/13,513.2229,跌
530,2330 台積電,2023/03/14,507.2552,漲
531,2330 台積電,2023/03/15,508.2498,持平
532,2330 台積電,2023/03/16,505.0000,漲
533,2330 台積電,2023/03/17,518.0000,漲
534,2330 台積電,2023/03/20,512.0000,漲
535,2330 台積電,2023/03/21,517.0000,漲
536,2330 台積電,2023/03/22,533.0000,0
537,2330 台積電,2023/03/23,538.0000,0
538,2330 台積電,2023/03/24,539.0000,0


In [7]:
# 更改年月日的日期格式
df_stock['年月日'] = pd.to_datetime(df_stock['年月日']).dt.date
df_stock.tail(10)

,證券代碼,年月日,收盤價(元),label
529,2330 台積電,2023-03-13,513.2229,跌
530,2330 台積電,2023-03-14,507.2552,漲
531,2330 台積電,2023-03-15,508.2498,持平
532,2330 台積電,2023-03-16,505.0000,漲
533,2330 台積電,2023-03-17,518.0000,漲
534,2330 台積電,2023-03-20,512.0000,漲
535,2330 台積電,2023-03-21,517.0000,漲
536,2330 台積電,2023-03-22,533.0000,0
537,2330 台積電,2023-03-23,538.0000,0
538,2330 台積電,2023-03-24,539.0000,0


In [8]:
upCnt = 0
downCnt = 0
flatCnt = 0
for i in range(len(df_stock)):
    if df_stock['label'][i] == '漲':
        upCnt += 1
    elif df_stock['label'][i] == '跌':
        downCnt += 1 
    else :
        flatCnt += 1
print('預估漲的天數：', upCnt)
print('預估跌的天數：', downCnt)
print('預估持平的天數：', flatCnt)

預估漲的天數： 180
預估跌的天數： 200
預估持平的天數： 159


In [9]:
# 丟掉持平的資料
df_stock = df_stock[df_stock.label != '持平']
df_stock = df_stock[df_stock.label != '0']
df_stock = df_stock.reset_index(drop = True)
df_stock

,證券代碼,年月日,收盤價(元),label
0,2330 台積電,2021-01-04,526.7744,漲
1,2330 台積電,2021-01-05,532.6711,漲
2,2330 台積電,2021-01-06,539.5506,漲
3,2330 台積電,2021-01-07,555.2752,漲
4,2330 台積電,2021-01-08,570.0170,漲
...,...,...,...,...
378,2330 台積電,2023-03-20,512.0000,漲
379,2330 台積電,2023-03-21,517.0000,漲
380,2330 台積電,2023-03-22,533.0000,0
381,2330 台積電,2023-03-23,538.0000,0


In [10]:
# import news excel 
df_news = pd.read_excel("../../TSMC_news_21-23.xlsx")
df_news.head()

,Unnamed: 0,post_time,title,content,token
0,0,2021-01-01,明年買哪幾檔晶片股?分析師首選 Nvidia 和 AMD,週四 (31 日) 華爾街分析師表示，Nvidia 和 AMD 本月漲勢落後於其他半導體類股...,"['週四', '日', '華爾街', '分析師', '表示', '和', '本', '月',..."
1,1,2021-01-01,《大陸產業》中芯成熟製程 獲美放行,【時報-台北電】遭美制裁的大陸晶圓代工龍頭中芯國際迎來曙光。業界傳出，中芯國際成熟製程獲得美...,"['時報', '台北電', '遭', '美', '制裁', '的', '大陸', '晶圓',..."
2,2,2021-01-04,《各報要聞》台積今年資本支出上看200億美元,【時報-台北電】晶圓代工龍頭台積電2020年繳出亮麗成績單，預期全年美元營收年成長率逾三成並...,"['時報', '台北', '電', '晶圓', '代工', '龍頭', '台積電', '年'..."
3,3,2021-01-04,《半導體》台積電等供應鏈力挺 聯發科Q1拚淡季不淡,【時報記者王逸芯台北報導】聯發科(2454)在去年第三季一舉超車高通，在行動晶片市占率衝上3...,"['時報', '記者', '王逸芯', '台北', '報導', '聯發科', '在', '去..."
4,4,2021-01-04,《半導體》2021年拚翻身 創意鎖漲停,【時報記者王逸芯台北報導】創意(3443)營運最壞時期已經過去，去年第四季在NRE案認列入帳...,"['時報', '記者', '王逸芯', '台北', '報導', '創意', '營運', '最..."


In [11]:
# 幫 news 標 label（用上面的日期去對照哪一天的 news 是漲/跌）
label_news = []
for i in range(len(df_news)):
    for j in range(len(df_stock)):
        if df_news['post_time'][i] == df_stock['年月日'][j]:
            label_news.append(df_stock['label'][j])
    if len(label_news) == i:
        label_news.append(0) # 日期沒有對到的情況（e.g. 週末） 

df_news['label'] = label_news
df_news.head(10)
df_news.tail(10)

/var/folders/mn/3mpydyqj1tq0wy92y446smg80000gn/T/ipykernel_11690/2265056801.py:5: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  if df_news['post_time'][i] == df_stock['年月日'][j]:


,Unnamed: 0,post_time,title,content,token,label
6822,6822,2023-03-20,算力大戰 台積日月光受惠,隨著微軟轉投資OpenAI推出的聊天機器人ChatGPT全球爆紅，包括Google、阿里巴巴...,"['隨著', '微軟', '轉', '投資', '推出', '的', '聊天', '機器人'...",漲
6823,6823,2023-03-20,銀行股再見殺氣，美股四巫日齊黑，道瓊急跌385點，微軟逆勢漲約1.2%,【財訊快報／陳孟朔】被華府接管的矽谷銀行(SVB)，其母公司SVB金融集團申請破產保護的同時...,"['財訊', '快報', '陳孟朔', '被', '華府', '接管', '的', '矽谷銀...",漲
6824,6824,2023-03-20,《熱門族群》外資釋出AI口袋名單股 台積電等3檔入列,【時報記者王逸芯台北報導】ChatGPT颳起AI旋風，美系外資針對AI產業也出具最新研究報告...,"['時報', '記者', '王逸芯', '台北', '報導', '颳起', '旋風', '美...",漲
6825,6825,2023-03-20,台積電蟬聯百大創新企業，專利與營業秘密雙軌保護成果，研發營收比8%,【財訊快報／記者李純君報導】台積電(2330)今年繼續蟬聯百大創新企業，其副法務長陳碧莉提到...,"['財訊快報', '記者', '李純君', '報導', '台積電', '今年', '繼續',...",漲
6826,6826,2023-03-20,台積電／台積電「創新」再獲獎 副法務長陳碧莉：去年在台、美專利獲准百發百中,台積電（2330）再次入選「2023全球百大創新機構獎」，出席領獎的台積電副法務漲陳碧莉表示...,"['台積電', '再次', '入選', '全球', '百', '大', '創新', '機構獎...",漲
6827,6827,2023-03-21,晶片業起死回生？日媒揭實際庫存真相：反轉訊號來了,半導體景氣反轉向下，從「供不應求」轉變成「供應過剩」，市場關注產業景氣何時才會翻轉向上，日媒...,"['半導體', '景氣', '反轉', '向', '下', '從', '供', '不', '...",漲
6828,6828,2023-03-21,《盤前掃瞄-基本面》高通加速轉單台灣；國發會估景氣4月探底,【時報-台北電】基本面：1.前一交易日新台幣以30.593元兌一美元收市，貶值3.9分，成交...,"['時報', '台北', '電', '基本面', '前', '一', '交易日', '新台幣...",漲
6829,6829,2023-03-21,《各報要聞》高通去中化 加速轉單台灣,【時報-台北電】為了因應半導體市場出現美國陣營及中國陣營的兩極化地緣政治風險，手機晶片大廠高...,"['時報', '台北電', '為了', '因應', '半導體', '市場', '出現', '...",漲
6830,6830,2023-03-21,《科技》台積電兩招…擴大專利版圖,【時報-台北電】晶圓代工龍頭台積電20日獲頒2023年科睿唯安全球百大創新機構獎，副法務長陳...,"['時報', '台北', '電晶圓', '代工', '龍頭', '台積電', '日', '獲...",漲
6831,6831,2023-03-21,《熱門族群》高通轉單喜訊 精測笑開懷、這兩檔卻擺臭臉,【時報-台北電】台股今日隨美股彈升，在金融股回神、AI等具題材族群續強下，大盤指數盤中彈升約...,"['時報', '台北電', '台股', '今日', '隨', '美股', '彈升', '在'...",漲


In [12]:
# 刪掉 label 是 0 的資料
df_news_no_zero = df_news[df_news.label != 0]
df_news_no_zero = df_news_no_zero.reset_index(drop = True)

In [20]:
df_news_up = df_news_no_zero[df_news_no_zero['label'] == '漲'] 
df_news_down = df_news_no_zero[df_news_no_zero['label'] == '跌'] 

In [24]:
len(df_news_down)

2286

In [16]:
import csv
import monpa
from monpa import utils
from collections import Counter
import math
import re

In [22]:
#  用來將字串以正則化處理去除中文字元以外的字元
def clearSentence(sentence):
    return re.sub(r'[^\u4e00-\u9fa5]+', '', sentence)

#  去除繁體中文常用詞
with open('./stopwords_zh.txt', 'r', encoding='utf-8') as file:
    stopwords = file.read().splitlines()
file.close()

In [25]:
tf_counter_up = Counter() #預備統計tf用
df_counter_up = Counter() #預備統計df用

upstr_list = []

for i in range(len(df_news_up)):
  df_tmp_up=Counter() #暫存本篇df用
  sentence_list = utils.short_sentence(df_news_up['content'].iloc[i]) #斷句
  tokenStr = str()
  for sentence in sentence_list:
    sentence_list2 = utils.short_sentence(sentence) 
    for item in sentence_list2:
        result_cut = monpa.cut(item) #斷詞
        for term in result_cut:
          term=term.strip() #去除前後多餘空白
          if(len(term) > 1): #若詞長>1
            tf_counter_up[term] += 1 #tf加1
            upstr_list.append(term)
          if(df_tmp_up[term] == 0): #若本篇之前不曾出現
            df_tmp_up[term] = 1 #df標為1
  df_counter_up += df_tmp_up # 累加多篇df

In [45]:
tfidf_up = Counter()
for i in range(len(upstr_list)):
    item  = upstr_list[i]
    pts = (1 + math.log(tf_counter_up[item]) * math.log(len(df_news_no_zero)/df_counter_up[item]))
    tfidf_up[item] = pts
pts

14.45124497576976

In [83]:
tf_counter_down = Counter() #預備統計tf用
df_counter_down = Counter() #預備統計df用

downstr_list = []

for i in range(len(df_news_down)):
  df_tmp_down=Counter() #暫存本篇df用
  sentence_list = utils.short_sentence(df_news_down['content'].iloc[i]) #斷句
  tokenStr = str()
  for sentence in sentence_list:
    sentence_list2 = utils.short_sentence(sentence) 
    for item in sentence_list2:
        result_cut = monpa.cut(item) #斷詞
        for term in result_cut:
          term=term.strip() #去除前後多餘空白
          if(len(term) > 1): #若詞長>1
            tf_counter_down[term] += 1 #tf加1
            downstr_list.append(term)
          if(df_tmp_down[term] == 0): #若本篇之前不曾出現
            df_tmp_down[term] = 1 #df標為1
  df_counter_down += df_tmp_down # 累加多篇df

In [86]:
tfidf_down = Counter()
for i in range(len(downstr_list)):
    item  = downstr_list[i]
    pts = (1 + math.log(tf_counter_down[item]) * math.log(len(df_news_no_zero)/df_counter_down[item]))
    tfidf_down[item] = pts
pts

22.48156416983101

In [87]:
allstr_list = upstr_list + downstr_list
tf_counter_all = Counter()
df_counter_all = Counter()
for item in allstr_list:
    tf_counter_all[item] = tf_counter_up[item] + tf_counter_down[item]
    df_counter_all[item] = df_counter_up[item] + df_counter_down[item]

In [88]:
#tfidf_chi_up
chi_tfidf_up = Counter()
for item in upstr_list:
    expected_tf = tf_counter_all[item] / (len(df_news_up.index) + len(df_news_down.index)) * len(df_news_up)
    expected_df = df_counter_all[item] / (len(df_news_up.index) + len(df_news_down.index)) * len(df_news_up)
    
    if tf_counter_up[item] < expected_tf:
        n1 = -1
    else:
        n1 = 1
    tf_pts = (tf_counter_all[item] - expected_tf)**2 / expected_tf
    if df_counter_up[item] < expected_df:
        n2 = -1
    else:
        n2 = 1
    df_pts = (df_counter_all[item] - expected_df)**2 / expected_df
    chi_pts = tfidf_up[item] * math.sqrt(tf_pts) * math.sqrt(df_pts) * n1 * n2
    chi_tfidf_up[item] = chi_pts

In [89]:
for term in chi_tfidf_up.most_common(100): #印出看漲 tfidf 前100名
  print(term[0],term[1])

台積電 49386.16549118778
晶片 37719.48864778843
美國 37395.481837708656
台股 34870.65345021222
市場 33408.344870283596
股價 29371.982021442953
奈米 28478.646975358795
產業 27810.373611655315
今年 27353.254304223774
製程 26893.748694720536
全球 26762.28986920426
晶圓 25633.79154887285
成長 25604.32721681808
預期 25445.21289584222
公司 24040.46701377504
需求 23844.30701669659
持續 23708.09052734583
外資 23695.257601712932
技術 21501.325476839807
上漲 20897.322320523363
客戶 20827.34056141571
中國 20766.226469113994
代工 19914.614400199935
可能 19284.78978597627
下跌 19273.553181515665
蘋果 19132.525465110117
報導 19109.922915586787
產品 19064.977411991662
法人 18631.117746713084
英特爾 18174.42474391786
生產 17985.769109887995
科技 17974.46747435318
目前 17509.991072496185
手機 16747.096579212874
主要 16697.13219004285
2022年 16551.275632066623
股市 16342.124850313327
設計 16206.705061896262
認為 15889.011375628692
企業 15512.510622829406
台北 15482.869977451666
IC 15441.64692024398
國際 15425.183858313347
相關 15306.551883656835
價格 15140.0582602351
族群 15095.527502747897
疫

In [90]:
#tfidf_chi_down
chi_tfidf_down = Counter()
for item in downstr_list:
    expected_tf = tf_counter_all[item] / (len(df_news_up.index) + len(df_news_down.index)) * len(df_news_down)
    expected_df = df_counter_all[item] / (len(df_news_up.index) + len(df_news_down.index)) * len(df_news_down)

    if tf_counter_down[item] < expected_tf:
        n1 = -1
    else:
        n1 = 1
    tf_pts = (tf_counter_all[item] - expected_tf)**2 / expected_tf
    if df_counter_down[item] < expected_df:
        n2 = -1
    else:
        n2 = 1
    df_pts = (df_counter_all[item] - expected_df)**2 / expected_df
    chi_pts = tfidf_down[item] * math.sqrt(tf_pts) * math.sqrt(df_pts) * n1 * n2
    chi_tfidf_down[item] = chi_pts

In [91]:
for term in chi_tfidf_down.most_common(100): #印出看跌 tfidf 前100名
  print(term[0],term[1])

台積電 27113.875402951493
美國 21445.078791626278
台股 21292.992304259285
晶片 20442.63645420771
市場 19186.89719909472
股價 17809.406289775092
奈米 17149.655349431185
製程 16584.337151061725
產業 16316.605453376711
晶圓 15893.19763412192
全球 15891.094628117022
今年 15647.82491811599
預期 15573.870381679437
成長 15338.779193338758
外資 14951.511324005633
需求 14328.997577049093
公司 14215.324803164998
持續 14056.262668887919
技術 12936.884291469845
中國 12783.384368105515
上漲 12750.04401577807
代工 12550.921808964733
客戶 12437.095991126858
可能 12132.549372597552
下跌 11827.875491314846
蘋果 11638.006079767496
報導 11619.177571734046
法人 11551.163130666324
產品 11506.113821276569
英特爾 11121.420139480428
生產 10996.236675052383
科技 10919.537996751282
2022年 10636.250049016162
目前 10598.507038888887
股市 10080.193672401603
主要 10058.251620260387
手機 10012.470449172219
認為 9941.584263083929
台北 9773.60606030665
設計 9737.267458114746
IC 9532.631100108336
價格 9471.958371012426
企業 9457.857646175196
族群 9454.281896409959
通膨 9293.056090196002
三星 9267.87500712550